In [1]:
from cfg import *
import PrimaryLibrary as PL
imp.reload(PL)
MesoPred = PL.MesoconnectomePredictor()

/home/ntimonidis/anaconda2/lib/python2.7/site-packages/nilearn/__init__.py:73: DeprecationWarning: Python2 support is deprecated and will be removed in the next release. Consider switching to Python 3.6 or 3.7.
  _python_deprecation_warnings()


importing Jupyter notebook from PrimaryLibrary.ipynb
importing Jupyter notebook from PrimaryLibrary.ipynb


In [133]:
def ParseTheMarkers(infolder = 'disease_gene_markers', infile = 'alzheimer_genes.csv'):
    startline = 0
    markers = []
    if infile == 'alzheimer_genes.csv':
        startline = 4
    with open(os.path.join(infolder,infile),'r') as fp:
        tbl = csv.reader(fp)
        for idx,val in enumerate(tbl):
            if idx >= startline:
                if type(val) is list and len(val) > 0:
                    for val_sub in val[0].split(', '):
                        markers.append(val_sub)
                elif type(val) is str and len(val) > 0:        
                    markers.append(val[0])
    return markers                    

In [ ]:
# New additions 25-05-2020: update Component summary by including 
#                          a) spectral clustering of H and then nmi with the allen annotation
#                          b) pearson correlation with the average template volume
#a)
from sklearn.cluster import spectral_clustering, SpectralClustering
from sklearn.metrics import normalized_mutual_info_score as nmi    

def PostHocLICA(annotation200, Non_zero_voxels, infile = 'data/Outputs_O_param_source_vis_all/M.p',
                maskfile = 'saves/masks_4_wild_type_vis_all.pkl',
                tempfile = 'average_template_200.nii.gz',
                comp_filter = None):

    M = pk.load(open(infile,'rb'))
    new_masks = pk.load(open(maskfile,'rb'))
    avg_template = nib.load(tempfile).get_data()
    lica2gray = np.zeros((len(M['H'])))
    flat_avg_temp = np.ndarray.flatten(avg_template,order = 'F')
    flat_annot = np.ndarray.flatten(annotation200[0],order = 'F')
    
    if comp_filter is None:
        comp_filter = np.arange(len(M['H']))
    
    affinity_2d = np.corrcoef(M['H'][comp_filter,:].T)
    spect_clus  = SpectralClustering(n_clusters=20, affinity = 'precomputed').fit(np.abs(affinity_2d))
    labels      = spect_clus.labels_
    nmi_score   = nmi(flat_annot[Non_zero_voxels][new_masks['vox_mask']],labels)
    print np.shape(labels), nmi_score

    flat_avg_temp_sub = flat_avg_temp[Non_zero_voxels][new_masks['vox_mask']]
    for i in comp_filter:
        lica2gray[i] = sci.stats.pearsonr(np.abs(M['H'][i,:]),flat_avg_temp_sub)[0]

    plt.hist(lica2gray)    


    return lica2gray, labels, nmi_score

In [ ]:
def CheckMarkers(gene_list, comps_of_int, infile = 'data/Outputs_O_param_200_source_vispor/'):
    
    # Document the z-statistics of a list of genes for a list of flicas
    M = pk.load(open(os.path.join(infile, 'M.p'),'rb'))
    gene_coef =  M['X'][0]
    gene_coef_z = StandardScaler().fit_transform(gene_coef)
    Dev_dict = {}
    for comp in comps_of_int:
        Dev_dict[comp] = {}
        for gene in gene_list:
            gene_id = [idx for idx,val in enumerate(MesoPred.params['Gene Acronyms Original']) if val == gene]
            try:
                Dev_dict[comp][gene] = gene_coef_z[gene_id[0],comp]
                if len(gene_id) > 0 and np.abs(gene_coef_z[gene_id[0],comp]) > 1:
                    #print(comp, gene, gene_coef_z[gene_id[0],comp])
                    a = 1
            except:
                Dev_dict[comp][gene] = 0
                
    return Dev_dict            

In [ ]:
def MarkNeighborhood(InputArray,n_zero_vox, thickness):
    
    input_shape = np.shape(InputArray)
    x_ind = np.arange(n_zero_vox[0]-thickness,n_zero_vox[0]+thickness+1)
    x_ind = x_ind[np.where(x_ind >= 0)[0]]
    x_ind = x_ind[np.where(x_ind < input_shape[0])[0]]
    y_ind = np.arange(n_zero_vox[1]-thickness,n_zero_vox[1]+thickness+1)
    y_ind = y_ind[np.where(y_ind >= 0)[0]]
    y_ind = y_ind[np.where(y_ind < input_shape[1])[0]]
    z_ind = np.arange(n_zero_vox[2]-thickness,n_zero_vox[2]+thickness+1)
    z_ind = z_ind[np.where(z_ind >= 0)[0]]
    z_ind = z_ind[np.where(z_ind < input_shape[2])[0]]
    neigh_mask = [x_ind,y_ind,z_ind]
    res_mask = np.meshgrid(neigh_mask[0],neigh_mask[1],neigh_mask[2])
    res_mask = np.reshape(res_mask,(3,np.shape(res_mask)[1]*np.shape(res_mask)[2]*np.shape(res_mask)[3]), order = 'F')
    
    return res_mask

In [ ]:
def VoxelThickening(InputArray, thickness = 1):
    input_shape = InputArray.shape
    n_zero_mask = np.nonzero(InputArray)
    print(len(np.nonzero(InputArray)[0])/(1.0*input_shape[0]*input_shape[1]*input_shape[2]))
    for vox_el in range(len(n_zero_mask[0])):
        n_zero_vox = (n_zero_mask[0][vox_el],n_zero_mask[1][vox_el],n_zero_mask[2][vox_el])
        neigh_mask = MarkNeighborhood(InputArray,n_zero_vox, thickness)
        try:
            InputArray[neigh_mask[0],neigh_mask[1],neigh_mask[2]] = 1
        except:
            print('Unexpected error! Lets debug')
            set_trace()
        
    print(len(np.nonzero(InputArray)[0])/(1.0*input_shape[0]*input_shape[1]*input_shape[2]))
    
    return InputArray    

In [142]:
def FindComponent(Gene_coeff, GeneSet, interval = 95, part_thr = 0.02):
    # find the component with the highest co-expression
    old_max_prc = -1; sel_component = -1
    low_perc = (100-interval)/2.0; high_perc = interval + low_perc

    for component in range(len(Gene_coeff[0])):
        sel_gene_footprint    = Gene_coeff[:,component]
        gene_high_thr = np.percentile(sel_gene_footprint, high_perc); gene_low_thr = np.percentile(sel_gene_footprint, low_perc)
        gene_grp = [idx for idx,val in enumerate(sel_gene_footprint) if val > gene_high_thr or val < gene_low_thr]
        gene_join = set(gene_grp).intersection(set(GeneSet))
        participation_prc = len(gene_join)/(1.0*len(GeneSet))
        print(component, len(GeneSet), len(gene_grp), len(gene_join))
        if participation_prc > part_thr and participation_prc > old_max_prc:
            sel_component =  component
            old_max_prc   =  participation_prc 
        #print(component, participation_prc)

    print('The component with the highest co-expression is: {}'.format(sel_component))
    
    return sel_component

In [ ]:
def Resample_Voxels(spatial_modules, resolution = 25, resolution2 = 100, modules = 'all', 
                    interpolation = 'continuous', savefile = 'saves/modules_res_200.nii.gz'):
    
    affine_res    = resolution*0.001*np.eye(4); affine_res[3,3] = 1.0
    affine_res2   = resolution2*0.001*np.eye(4); affine_res2[3,3] = 1.0
    annot_path    = '../25 3 2019/annotation/ccf_2017/annotation_{}.nrrd'.format(resolution)
    
    # first rearrange the array such that the feature space is in the last dimension
    input_size = spatial_modules.shape
    '''spatial_modules_res = np.zeros((input_size[1],input_size[2],input_size[3],input_size[0]))
    for tracer in range(input_size[0]):
        spatial_modules_res[:,:,:,tracer] = spatial_modules[tracer,:,:,:]'''
    
    annot_res     = nrrd.read(annot_path)
    annot_res_nii = nib.Nifti1Image(annot_res[0], affine_res)
    upsamp_shape  = annot_res[0].shape 
    modules_res   = np.zeros((upsamp_shape[0],upsamp_shape[1],upsamp_shape[2],np.shape(spatial_modules)[3]))
    
    
    if modules == 'all': 
        mod_set = range(np.shape(modules_res)[3])
    else:
        mod_set = [modules]
        
    start_time = time.time()
    for module in mod_set:    # iterate over each individual module
        print('module no {}'.format(module))
        modules200   = nib.Nifti1Image(spatial_modules[:,:,:,module], affine_res2)
        tmp  = resample_to_img(modules200, annot_res_nii, interpolation = interpolation)
        if len(mod_set) > 1:
            modules_res[:,:,:,module] = tmp.get_data()
        else:
            modules_res[:,:,:,0] = tmp.get_data()
        ratio_of_100 =  len(np.nonzero(spatial_modules[:,:,:,module])[0])/(1.0*input_size[0]*input_size[1]*input_size[2])
        ratio_of_200 =  len(np.nonzero(tmp.get_data())[0])/(1.0*tmp.shape[0]*tmp.shape[1]*tmp.shape[2])  
        print(ratio_of_100,ratio_of_200)
            
    end_time = time.time()    
    print('upsampling has been completed!\nTime elapsed: {}'.format((end_time - start_time)/60.0))
    
    modules_res_img = nib.Nifti1Image(modules_res, affine_res)
    nib.save(modules_res_img, savefile)
    
    return modules_res


In [ ]:
def Merge_GO_Meta(go_meta):
    
    tmp_list = go_meta.values()
    for r1,item1 in enumerate(tmp_list):
        s = item1
        for r2,item2 in enumerate(tmp_list):
            flag = 0
            intersection = set(item1.split(' ')).intersection(set(item2.split(' ')))
            union = set(list(set(item1.split(' '))) + list(set(item2.split(' '))))
            jac_sim = len(intersection)/(1.0*len(union))
            if jac_sim >= 0.7 and jac_sim < 1.0:
                difference = union - intersection
                for element in difference:
                    if element not in item1 and len(item1) > len(item2):
                        s+= ' and ' + str(element)
                        flag = 1
                if flag == 1: # new element has been placed:
                    tmp_list.pop(r2)
            if len(s) > len(item1):
                tmp_list.pop(r1)
                tmp_list.append(s)           

    s = ''
    cnt = 0
    for value in tmp_list:
        if cnt > 0:
            value = value.lower()
        s += str(value)
        if cnt < len(tmp_list)-1:
            s+= ', and '
        if cnt == len(tmp_list)-1:
            s+='.'
        cnt+=1            

    return s

def GetGroupEnrichments(GeneIds, masks, infile = 'data/Outputs_O_param_400_mouselight_mo/M.p', hgCutoff = 0.001,
                       comps_of_int = None, tail = 97.5, method = 'relative'):
    
    # Note: chosing the method to be relative performs enrichment analysis relative to the compared sets,
    # meaning that only genes that are exclusively highly expressed within a group will be selected.
    # The other option is 'absolute', leading to the classical enrichment analysis
    
    def ListDiff(List1,List2):
        # Returns elements from two lists, in separate new lists, that are exclusive to each other.
        DiffList = [val for val in List1 if val not in List2]
        DiffList2 = [val for val in List2 if val not in List1]

        print len(DiffList), len(List1)
        print len(DiffList2), len(List2)

        return DiffList, DiffList2

    M = pk.load(open(infile,'rb'))
    go_df = OrderedDict()
    GeneListDict = OrderedDict()
    inj_grp = infile.split('_')
    ing_grp = inj_grp[len(inj_grp)-1].split('/')[0]
    if comps_of_int == None: comps_of_int = np.arange(len(M['H']))
    UniverseList = [val for val in GeneIds if val is not None]

    for comp in comps_of_int:
        Component_res, spatial_footprint = ComponentSummary(M, sel_component = comp, \
                                                               tail = tail, add_mask = masks['vox_mask'])
        GeneListDict[comp]   = [GeneIds[val] for val in Component_res['gene indices'] 
                                 if GeneIds[val] is not None]

    go_df = OrderedDict()
    go_meta = OrderedDict()
    for comp in comps_of_int:
        new_key = str(ing_grp) + ' ICA ' + str(comp)
        GeneList2 = []
        for comp2 in comps_of_int:
            if comp != comp2:
                GeneList2.append(GeneListDict[comp2])
        GeneList2 = np.unique([val2 for val in GeneList2 for val2 in val])        
        DiffList, DiffList2 = ListDiff(GeneListDict[comp],GeneList2)
        if method == 'relative':
            go_df[new_key],go_meta[new_key] = MesoPred.GOenrichment(DiffList, UniverseList, hgCutoff)
            print new_key, len(DiffList)
        else:
            go_df[new_key], go_meta[new_key] = MesoPred.GOenrichment(GeneListDict[comp], UniverseList, hgCutoff)
            print new_key, len(GeneListDict[comp])

    Ontology_dict = OrderedDict()  
    unique_dict = OrderedDict()
    for key in go_df.keys():
        aggr_annots = np.unique([val for comp2 in go_df.keys()\
                                 for val in go_df[comp2]['Term'].values\
                                 if comp2 != key])
        unique_dict[key] = np.unique([val for idx,val in enumerate(go_df[key]['Term'].values)\
                                 if val not in aggr_annots])
        unique_ids = [idx for idx,val in enumerate(go_df[key]['Term'].values)\
                                 if val not in aggr_annots]
        Ontology_dict[key] = go_df[key].iloc[unique_ids, :]
        
    unique_df = pd.DataFrame.from_dict(unique_dict, orient = 'index').T   
    
    return Ontology_dict, go_meta, unique_df

In [1]:
def RemoveNaNs(InputArray, annot_path = '../25 3 2019/annotation/ccf_2017/',resolution = 200):
    
    annot_path    = os.path.join(annot_path,'annotation_{}.nrrd'.format(resolution))
    
    annot_res     = nrrd.read(annot_path)
    unique_annots = np.unique(annot_res[0])
    
    feature_size = np.shape(InputArray)[3] # last dimension of nifti-formated volumetric arrays
    
    for feature in range(feature_size):    # iterate over each individual module
        for annot in unique_annots:
            annotated_elements = InputArray[:,:,:,feature][annot_res[0] == annot]
            possible_nans = np.isnan(annotated_elements)
            InputArray[:,:,:,feature][annot_res[0] == annot][possible_nans] = np.nanmean(annotated_elements)
            
    return InputArray


In [ ]:
def GenerativePriors(InputMat, M, modality = 'voxels'):
    # Generating the coefficient matrices ...
    OutputMat = np.zeros(np.shape(InputMat))
    # options = {'voxels','weights','genes','tracers'}
    if modality == 'voxels':
        OutputMat  = np.random.normal(loc = 0, scale = 1, size = np.shape(InputMat))
    elif modality == 'weights':
        for comp in range(np.shape(InputMat)[1]):
            for mod in range(2):
                OutputMat[:,comp] = np.random.normal(loc = 0, scale = np.transpose(M['ARD'][mod,comp]), size = 1)
    else:
        if modality == 'genes':
            mod = 0 
        else:
            mod = 1

        for comp in range(np.shape(InputMat)[1]):
            for mix in range(3):
                OutputMat[:,comp] += M['pi'][mod][mix,comp]*np.random.normal\
                (loc = M['mu'][mod][mix,comp], scale = M['beta'][mod][mix,comp], size = np.shape(InputMat)[0])       

    return OutputMat

In [ ]:
def VisualizeVolume(InputMat, sel_element, savefile = None): 
    
    InputMat_25 =  Resample_Voxels(InputMat, resolution = 25, modules = sel_element)
    InputMat_10 =  Resample_Voxels(InputMat, resolution = 10, modules = sel_element)
    BP_10 = PL.BrainPlotter(resolution = 10)
    BP_10.fit(projection = InputMat_10[:,:,:,0])
    BP_10.plot_flatmap(savefile = savefile + '_flatmap')
    BP_25 = PL.BrainPlotter(resolution = 25)
    BP_25.fit(projection = InputMat_25[:,:,:,0])
    BP_25.plot_slice(savefile = savefile + '_slice')

In [ ]:
def GenerativePosteriors(InputMat, M, modality = 'voxels'):
    # Generating the coefficient matrices ...
    OutputMat = np.zeros(np.shape(InputMat))
    # options = {'voxels','weights','genes','tracers'}
    if modality == 'voxels':
        OutputMat  = np.random.normal(loc = 0, scale = 1, size = np.shape(InputMat))
    elif modality == 'weights':
        for comp in range(np.shape(InputMat)[1]):
            for mod in range(2):
                OutputMat[:,comp] = np.random.normal(loc = 0, scale = np.transpose(M['ARD'][mod,comp]), size = 1)
    else:
        if modality == 'genes':
            mod = 0 
        else:
            mod = 1

        for comp in range(np.shape(InputMat)[1]):
            for mix in range(3):
                OutputMat[:,comp] += M['pi'][mod][mix,comp]*np.random.normal\
                (loc = M['mu'][mod][mix,comp], scale = M['beta'][mod][mix,comp], size = np.shape(InputMat)[0])       

    return OutputMat

In [ ]:
#### Step-wise component ranking routine
def Step_Wise_Reconstruction(M,InputMat, mod_num = 0, thr = 0.9):
    comp_list = np.arange(0,len(M['H']))
    opt_subset = []
    flag = 0           # condition indicating when a suboptimal solution has been reached (1) or not (0)
    top_solution = -10
    ReconMat     = np.matmul(np.multiply(M['X'][mod_num],M['W'][mod_num]),M['H'])
    Error        = InputMat - ReconMat
    while comp_list.size > 0 and flag == 0:
        local_perf_list = []
        for indeks,element in enumerate(comp_list):
            temp_merge    = list(opt_subset); temp_merge.append(element)
            ReconMat      = np.matmul(np.multiply(M['X'][mod_num][:,temp_merge],
                                                  M['W'][mod_num][0,temp_merge]),M['H'][temp_merge,:])
            r2_tracer_err = r2_score(InputMat,ReconMat + Error)
            local_perf_list.append(r2_tracer_err)
        top_idx    = np.argmax(local_perf_list)    
        top_scorer = comp_list[top_idx]
        if np.max(local_perf_list) < top_solution:
            flag = 1 # termination condition no 2
        else:
            top_solution = np.max(local_perf_list) 
            opt_subset.append(top_scorer)
            comp_list = np.delete(comp_list,top_idx)
            if top_solution >= thr:
                print('optimal r2 has been reached')
                flag = 1

    return opt_subset,top_solution

In [ ]:
def ValidateTheResults(Data_Modality,ReconModality,ModalityError):
    r2s = []; MSEs = []; err_r2s = []; err_MSEs = []
    for mod in Data_Modality.keys():
        if mod >= len(ReconModality): continue
        Mod_Rec_Orig = (np.square(Data_Modality[mod].T - ReconModality[mod].T)).mean(axis=None)
        MSEs.append(Mod_Rec_Orig)
        r2_mod = r2_score(Data_Modality[mod].T, ReconModality[mod].T)
        r2s.append(r2_mod)
        Mod_Err_Orig = (np.square(Data_Modality[mod].T - ModalityError[mod].T)).mean(axis=None)
        err_MSEs.append(Mod_Err_Orig)
        r2_mod_err = r2_score(Data_Modality[mod].T, ModalityError[mod].T)
        err_r2s.append(r2_mod_err)
        
        print('Reconstruction to Error comparison modality {} (MSE): {} - {}'.format(mod, Mod_Rec_Orig, Mod_Err_Orig))
        print('Reconstruction to Error comparison modality {} (r2): {} - {}'.format(mod, r2_mod, r2_mod_err))
        
    return r2s, MSEs, err_r2s, err_MSEs

In [ ]:
def ReconstructingData(M,Spatial_map,mod_num):
  
    VoxelSize = len(Spatial_map)
    ReconMat  = np.matmul(np.multiply(M['X'][mod_num],M['W'][mod_num]),Spatial_map)
   
    return ReconMat

clear_output()
   

In [ ]:
def Predictingdata(M, Data_Modality, GeneError = None, TracerError = None):
    gene_coef   = np.multiply(M['X'][0],M['W'][0])
    tracer_coef = np.multiply(M['X'][1],M['W'][1])
    if GeneError is None:
        GeneError = np.zeros(np.shape(Data_Modality[0]))
    if TracerError is None:
        TracerError = np.zeros(np.shape(Data_Modality[1]))    
    H_hat       = (np.linalg.inv((gene_coef).T.dot(gene_coef)).dot(gene_coef.T)).dot(Data_Modality[0] - GeneError)
    Y_con_pred  = tracer_coef.dot(H_hat) + TracerError
    
    return Y_con_pred

In [19]:
def GetAncestor(structure_abbrev, level = None, mode = 'id'):
    mcc = MouseConnectivityCache(manifest_file='connectivity/mouse_connectivity_manifest.json')
    # getting the structure tree of the CCF v3
    structure_tree = mcc.get_structure_tree()

    with open('../CCP_Collab_Backup_2/structures.csv','rb') as fp:
        structure   = PL.readtable(fp) 

    # getting the ids of the source areas    
    if mode == 'acro':
        src_ids = [structure['id'][id2] for idx,val in enumerate(structure_abbrev) 
                       for id2,val2 in enumerate(structure['acronym']) if val == val2]
    elif mode == 'id':    
        src_ids = structure_abbrev
        
    # creating an ancestry dictionary
    try:
        ancestry_trg_dict = structure_tree.ancestors(map(int, src_ids))
    except:
        print('Id not included in the structure')
        
    primary_list = ['CTX', 'OLF', 'HIP', 'RHP', 'STR', 'PAL', 'TH', 'HY', 'MB', 'P', 'MY', 'CB']
    if level is not None:
        primary_src_acros = OrderedDict([(structure_abbrev[i],ancestry_trg_dict[i][level]['name']) for i in range(len(ancestry_trg_dict)) if len(ancestry_trg_dict[i]) > level])
    else:
        primary_src_acros = OrderedDict() #[(structure_abbrev[i],ancestry_trg_dict[i][level]['name']) for i in range(len(ancestry_trg_dict)) for level in range(len(ancestry_trg_dict[i])) if ancestry_trg_dict[i][level]['acronym'] in primary_list])
        for i in range(len(ancestry_trg_dict)):
            for level in range(len(ancestry_trg_dict[i])):
                if ancestry_trg_dict[i][level]['acronym'] in primary_list:
                    primary_src_acros[structure_abbrev[i]] = ancestry_trg_dict[i][level]['name']
                    #print(i, ancestry_trg_dict[i][level]['acronym'])      
                    break
                    
    return primary_src_acros

In [7]:
# set parameters for picking tailed samples
def ComponentSummary(M, Data_Modality = None, mask_path = 'data/annotation200_mask.nii.gz', sel_component = 0, 
                     tail = 99, resolution = 200, nifti = 'no', add_mask = None):
    
    # Initialization of important variables 
    high_perc = 100 - (100 - tail)/2; low_perc = (100 - tail)/2
    with open('../CCP_Collab_Backup_2/structures.csv','rb') as fp:
        structure       = PL.readtable(fp) 
        struct_idx_dict = {val:idx for idx,val in enumerate(structure['id'])}
    
    annot_path = '../25 3 2019/annotation/ccf_2017/annotation_{}.nrrd'.format(resolution)
    n_mask = nib.load(os.path.join(mask_path)).get_data() 
    mask2d = np.reshape(n_mask,[n_mask.shape[0]*n_mask.shape[1]*n_mask.shape[2],1], order='F')
    Non_zero_voxels = ~np.all(mask2d == 0, axis = 1)
    if add_mask is not None:
        new_nzero = np.nonzero(Non_zero_voxels == True)[0][add_mask]
        Non_zero_voxels = [True if idx in new_nzero else False for idx,val in enumerate(Non_zero_voxels)]
    annotation = nrrd.read(annot_path)
    annot_shape = n_mask.shape
    BP = PL.BrainPlotter(resolution = resolution)
    mcc = MouseConnectivityCache(resolution = resolution)
    #annotation_parents = BP.ReduceToParent(annotation[0], mcc)
    #annot_parent_res  = np.reshape(annotation[0],(np.shape(annotation[0])[0]*np.shape(annotation[0])[1]*np.shape(annotation[0])[2]),order='F')
    
    # select participations of the modality of interest and find the proper cutoffs
    annot_res         = np.reshape(annotation[0],
                                       (annot_shape[0]*annot_shape[1]*annot_shape[2]), order='F')
    annot_nzero       = annot_res[Non_zero_voxels]
    sel_spatial_footprint = M['H'][sel_component,:]
    sel_gene_footprint    = M['X'][0][:,sel_component]
    sel_tracer_footprint  = M['X'][1][:,sel_component] 
    
    # Scale Arrays by multiplying them with their components
    spatial_footprint = {} 
    for i in range(len(M['X'])+1):
        if Data_Modality is None: break
        if nifti == 'no': break
        if i < 2:
            tmp     = np.dot(M['X'][i][:,sel_component].T,Data_Modality[i]).T
            if add_mask is not None:
                tmp = tmp[add_mask]
            outfile = 'saves/modality_{}_component_{}'.format(i, sel_component)
        else:
            tmp     = np.reshape(M['H'][sel_component,:], (len(M['H'][0]),1))
            outfile = 'saves/H_of_component_{}'.format(sel_component)
        
        spatial_footprint[i] = np.zeros(mask2d.shape)
        spatial_footprint[i][Non_zero_voxels] = tmp
        spatial_footprint[i] = np.reshape(spatial_footprint[i],
                                          (annot_shape[0],annot_shape[1],annot_shape[2],1), 
                                          order = 'F')
        PL.Save2Nifti(spatial_footprint[i], outfile = outfile)     
        VisualizeVolume(abs(spatial_footprint[i]), 0, 
                        savefile = 'component_{}_map'.format(sel_component))
     
    gene_high_thr = np.percentile(sel_gene_footprint, high_perc); 
    gene_low_thr = np.percentile(sel_gene_footprint, low_perc)
    tracer_high_thr = np.percentile(sel_tracer_footprint, high_perc); 
    tracer_low_thr = np.percentile(sel_tracer_footprint, low_perc)
    spatial_high_thr = np.percentile(sel_spatial_footprint, high_perc); 
    spatial_low_thr = np.percentile(sel_spatial_footprint, low_perc)
    
    # stratify the groups belonging to two tails
    spatial_grp = [idx for idx,val in enumerate(sel_spatial_footprint) 
                   if val >= spatial_high_thr or val <= spatial_low_thr]
    gene_grp = [idx for idx,val in enumerate(sel_gene_footprint) 
                if val >= gene_high_thr or val <= gene_low_thr]
    tracer_grp = [idx for idx,val in enumerate(sel_tracer_footprint) 
                  if val >= tracer_high_thr or val <= tracer_low_thr]    

    
    # Link the activated voxels to coarse-grain target brain areas
    target = {}
    if np.max(spatial_grp) > 428:
        strong_vxls = np.argsort(np.abs(M['H'][sel_component,:]))[::-1][0:20]
        strong_vxls_annot = [str(annot_nzero[val]) for val in strong_vxls]
        target['strong areas'] = np.unique([structure['acronym'][idx] for idx,val in enumerate(structure['id']) for val2 in strong_vxls_annot if val==val2])
        target['annot'] = np.unique([str(annot_nzero[val]) for val in spatial_grp])
    else:
        target['annot'] = [structure['id'][MesoPred.params['remaining_indices'][val]] 
                           for val in spatial_grp]
     
    target['acro'] = np.unique(GetAncestor(target['annot']))
    target['leaf acro'] = np.unique([structure['acronym'][idx] for idx,val in enumerate(structure['id']) for val2 in target['annot'] if val==val2])
    unique_targets = np.unique(target['acro'][0].values(), return_counts = True)
    target['acro'] = {v1:v2 for v1,v2 in zip(unique_targets[0],unique_targets[1])}
    target_only_ancestor = target['acro'].keys()
    target['idx']   = [struct_idx_dict[str(val)] for val in target['annot'] 
                       if str(val) in struct_idx_dict.keys()]

    # Link the selected injections to their source brain areas
    source = np.unique([MesoPred.ConDict['wild_type']['structure-abbrev'][val] for val in tracer_grp])
    #source = [structure['name'][id2] for val in source for id2,val2 in enumerate(structure['acronym']) if val == val2]
    source_only_ancestor = [GetAncestor([val],0, mode = 'acro') for val in source]
    source_only_ancestor = np.unique([val.values()[0] for val in source_only_ancestor if len(val.values()) > 0])
    
    #Link the gene groups to significant ontologies
    gene_id   = [MesoPred.params['Gene Ids Original'][val] for val in gene_grp 
                 if MesoPred.params['Gene Ids Original'][val] is not None]
    gene_acro = [MesoPred.params['Gene Acronyms Original'][val] for val in gene_grp 
                 if MesoPred.params['Gene Acronyms Original'][val] is not None]
    #ontology  = MesoPred.GOenrichment(gene_id)

    Component_res = {'source areas': source_only_ancestor, 'target areas': target['acro'],
                    'target indices': spatial_grp, 'source indices': tracer_grp, 'gene acronyms': 
                     gene_acro, 'gene indices': gene_grp, 'target annots': target['annot'],
                    'strongest areas': target['strong areas'], 'leaf acronyms': target['leaf acro']}

    GO_DF = pd.DataFrame(dict([(k, pd.Series(v)) for k,v in Component_res.items() 
                               if 'indices' not in k and 'source areas' not in k]))
    GO_DF.to_csv('figures/component_{}_new.csv'.format(sel_component))

    return Component_res, spatial_footprint

In [10]:
def HeatmapVisualization(inputMat, index = None, columns = None, savefig = None):
    
    if index is not None and columns is not None:
        #NeuronDf = pd.DataFrame(np.power(inputMat, 1/3.0),index = index, columns = columns) 
        NeuronDf = pd.DataFrame(inputMat,index = index, columns = columns) 
    else:    
        NeuronDf = pd.DataFrame(np.power(inputMat, 1/3.0))
        
    plt.figure(figsize=(20,12))
    sns.heatmap(NeuronDf)
    ax = plt.gca()
    ax.xaxis.set_tick_params(labelsize = 12, rotation = 280)
    ax.yaxis.set_tick_params(labelsize = 12)
    if savefig is not None:
        plt.savefig(savefig)
    plt.show()

In [2]:
def VolumeBuilder(Data_Modality, Non_zero_voxels, comp_set, shape_of_200, M = None, 
                  outfile = 'saves', filename = 'modality_', mod_list = None, masks = None):
    
    Volume = OrderedDict()
    affine_res = np.eye(4)*0.2; affine_res[3,3] = 1
    if mod_list == None: mod_list = list(Data_Modality.keys())
    for mod in mod_list: # Pick a data modality
        spatial_map_res = np.zeros((shape_of_200[0],shape_of_200[1],shape_of_200[2], len(comp_set)))
        for i,dicti in enumerate(comp_set):
            spatial_map_tmp = np.zeros((shape_of_200[0]*shape_of_200[1]*shape_of_200[2],1))
            if M is not None:
                if masks is not None and mod == 1:
                    A = Data_Modality[mod][masks['source_mask'],:]
                    spatial_map_tmp[np.nonzero(Non_zero_voxels)[0][masks['vox_mask']]] = \
                    np.matmul(M['X'][mod][:,dicti]*M['W'][mod][0][dicti], A[:,masks['vox_mask']]).T
                elif masks is not None and mod == 2:    
                        B = Data_Modality[mod][masks['source_mask_II'],:]
                        spatial_map_tmp[np.nonzero(Non_zero_voxels)[0][masks['vox_mask']]] = \
                    np.matmul(M['X'][mod][:,dicti]*M['W'][mod][0][dicti], B[:,masks['vox_mask']]).T
                else:    
                    spatial_map_tmp[np.nonzero(Non_zero_voxels)[0]] = np.matmul(M['X'][mod][:,dicti]*M['W'][mod][0][dicti],\
                                                                            Data_Modality[mod]).T

            else:
                spatial_map_tmp[np.nonzero(Non_zero_voxels)[0],0] = Data_Modality[mod][dicti,:].T
            spatial_map_res[:,:,:,i] = np.reshape(spatial_map_tmp,
                                         (shape_of_200[0],shape_of_200[1],shape_of_200[2]), order = 'F')

        spatial_map_nii = nib.Nifti1Image(spatial_map_res, affine_res)
        print(mod, spatial_map_nii.shape)
        nib.save(spatial_map_nii,os.path.join(outfile, filename + '_{}.nii.gz'.format(mod))) 

In [4]:
def PreProcessing(InputData, mask_path = 'data/annotation200_mask.nii.gz', analysis = 'volumetric',
                  add_mask = None):
    
    import flica_various as alb_various
    
    n_mask = nib.load(os.path.join(mask_path)).get_data() 
    mask2d = np.reshape(n_mask,[n_mask.shape[0]* n_mask.shape[1]* n_mask.shape[2],1], order='F')
    Non_zero_voxels = ~np.all(mask2d == 0, axis=1)
    
    Data_Modality = {}; cnt = 0
    Data_Modality_sc = {}
    #Data_Modality[0] = ExpPerInj; Data_Modality[1] = ProjPerInj
    for modality in InputData:
        Data_Modality[cnt] = modality
        cnt += 1
    
    for k in Data_Modality.keys():
        shape = Data_Modality[k].shape
        if analysis == 'volumetric' and len(shape) > 2:
            Data_Modality[k] = np.reshape(Data_Modality[k] ,[shape[0]*shape[1]*shape[2],shape[3]], order='F')
            Data_Modality[k] = Data_Modality[k][Non_zero_voxels,:].T
            # Mew step for reduced masks
            if add_mask is not None:
                Data_Modality[k] = Data_Modality[k][:,add_mask['vox_mask']]
                if len(Data_Modality[k]) < 1000: # injection matrix
                    Data_Modality[k]  = Data_Modality[k][add_mask['source_mask'],:]
        Data_Modality_sc[k] = Data_Modality[k]- (np.matrix(np.mean(Data_Modality[k],1)).T)   
        scaling_data_transform = alb_various.rms(Data_Modality_sc[k],[],[])       
        Data_Modality_sc[k] = np.divide(Data_Modality_sc[k],scaling_data_transform)
        if analysis == 'unionized':
            Data_Modality[k] = Data_Modality[k].T
            
    return Data_Modality, Data_Modality_sc, Non_zero_voxels        

In [ ]:
def BuildWhiteMatterMask(annotation200):
    
    unique_annot = np.unique(annotation200[0])
    unique_annot = np.delete(unique_annot,0)
    ancestry_results = GetAncestor(unique_annot, level = 3)
    fiber_ids =  [key for key,value in ancestry_results.items() if value == 'fiber tracts']
    fiber_mask = np.zeros(np.shape(annotation200[0])) + 1
    for i,x in enumerate(annotation200[0]):
        for j,y in enumerate(x):
            for k,z in enumerate(y):
                if z in fiber_ids:
                    fiber_mask[i,j,k] = 0
    affine_res = np.eye(4)*0.2; affine_res[3,3] = 1.0
    fiber_mask_nii = nib.Nifti1Image(fiber_mask, affine_res)    
    nib.save(fiber_mask_nii, 'fiber_mask.nii.gz')
    
    return fiber_mask

In [ ]:
def ConnectivitySim(Data_Modality, M, lamda, gene_list, mask_path = None, savefile = None, sel_component = None):
    
    # Given multiple data modalities and a dictionary of mixing components, this function predicts changes in 
    # modality 2 given changes in a modality 1, finds the features with the strongest changes and compares 
    # them to their mixing coefficients
    
    # Initialize the Delta G matrix 
    Delta_G = np.zeros(np.shape(Data_Modality[0]))
    
    # Reconstruct Gene expression and connectivity arrays based on the factorized modules
    ReconGene   = ReconstructingData(M,M['H'],0)
    ReconTracer = ReconstructingData(M,M['H'],1)

    # Estimate the residuals based on difference between the original and reconstructed arrays  
    GeneError   = Data_Modality[0] - ReconGene
    TracerError = Data_Modality[1] - ReconTracer
    
    # Alter the gene expression patterns 
    Delta_G[gene_list,:] = 1 # Modifying the genes-columns of interest
    Data_Modality_new = copy.deepcopy(Data_Modality) 
    #Data_Modality_new[0] = (1-lamda)*Data_Modality[0] + lamda*Delta_G
    Data_Modality_new[0] = np.multiply(Data_Modality[0].T,1-lamda).T + np.multiply(Delta_G.T,lamda).T
    
    # Predict structural connectivity based on altered gene expression patterns
    Data_Modality_new[1]  = Predictingdata(M, Data_Modality_new, GeneError, TracerError)
    
    # Evaluate differences between the actual and the predicted connectivity patterns
    con_effects = np.linalg.norm((Data_Modality_new[1]  - Data_Modality[1]), axis = 1)
    
    # Reshape the predicted connectivity array into a 4D array for NifTI visualization
    new_mat = []
    spatial_footprint_pert = {}
    if mask_path is not None and savefile is not None:
        n_mask = nib.load(os.path.join(mask_path)).get_data() 
        mask2d = np.reshape(n_mask,[n_mask.shape[0]* n_mask.shape[1]* n_mask.shape[2],1], order='F')
        Non_zero_voxels = ~np.all(mask2d == 0, axis=1)
        new_con = np.zeros((n_mask.shape[0],n_mask.shape[1],n_mask.shape[2],len(Data_Modality[1])))

        for tracer in range(len(Data_Modality_new[1])):
            tmp = np.zeros(mask2d.shape)
            tmp[Non_zero_voxels] = Data_Modality_new[1] [tracer,:].T
            new_con[:,:,:,tracer] =  np.reshape(tmp,(n_mask.shape[0],n_mask.shape[1],n_mask.shape[2],1), order = 'F')[:,:,:,0]
        if savefile is not None: PL.Save2Nifti(new_con, outfile = savefile)   
        
        for i in range(len(M['X'])):
            if sel_component is None: continue
            tmp     = np.dot(M['X'][i][:,sel_component].T, Data_Modality_new[i]).T
            outfile = 'saves/modality_{}_pertubed_{}_comp_{}'.format(i, lamda, sel_component)
            spatial_footprint_pert[i] = np.zeros(mask2d.shape)
            spatial_footprint_pert[i][Non_zero_voxels] = tmp
            spatial_footprint_pert[i] = np.reshape(spatial_footprint_pert[i],
                                          (n_mask.shape[0],n_mask.shape[1],n_mask.shape[2],1), order = 'F')
            
            PL.Save2Nifti(spatial_footprint_pert[i], outfile = outfile) 
               

    return Data_Modality_new,con_effects, spatial_footprint_pert

In [15]:
def Genes_To_Brain(keyword):
    
    filenames = os.listdir('saves/')
    filenames = [filename for filename in filenames if '.xlsx' in filename]
    # Searching or .xlsx files for parsing cell-type and anatomical genetic markers 
    
    OutDict = OrderedDict() # initialization of the arrays
    
    if 'Genes_To_Brain_Link.xlsx' in filenames and 'allen_difexp' in keyword:
        wb = load_workbook('saves/Genes_To_Brain_Link.xlsx')
        OutDict = OrderedDict()
        str_list = []
        for key in wb.get_sheet_names():
            ws = wb[key]
            new_key = key.split('_')[0]
            #OutDict[new_key] = OrderedDict()
            for i,row in enumerate(ws.rows): 
                if i == 0: 
                    for j in range(4,len(row)):
                        if row[j].value not in str_list:
                            str_list.append(row[j].value)       
                if i > 0: 
                    if row[0].value not in OutDict.keys():
                        OutDict[row[0].value] = OrderedDict()
                    for j in range(3,len(row)):
                        OutDict[row[0].value][str_list[j-4]] = row[j].value
                        
    
    if 'Tasic_2015_Supplementary_Markers.xlsx' in filenames and 'tasic_15' in keyword:
        wb = xlrd.open_workbook('saves/Tasic_2015_Supplementary_Markers.xlsx')
        ws = wb.sheet_by_index(0)
        OutDict = {}
        for row_id in range(ws.nrows):
            row =  ws.row(row_id) # get row elements
            if row_id == 0: # do the keys
                OutDict = OrderedDict((r.value, []) if r.value != '' else ('Transcriptomic subtype',[]) for r in row)
                key_list = OutDict.keys()
            else:  # put items in the corresponding keys
                for i,r in enumerate(row): OutDict[key_list[i]].append(r.value)
                    
    if 'Supplementary_Table_9_Cell_types_markers.xlsx' in filenames and 'tasic_19' in keyword:
        wb = xlrd.open_workbook('saves/Supplementary_Table_9_Cell_types_markers.xlsx')
        ws = wb.sheet_by_index(0)
        OutDict = {}
        for row_id in range(ws.nrows):
            row =  ws.row(row_id) # get row elements
            if row_id == 0: # do the keys
                OutDict = OrderedDict((r.value, []) for r in row)
                key_list = OutDict.keys()
            else:  # put items in the corresponding keys
                for i,r in enumerate(row): OutDict[key_list[i]].append(r.value)
                    
        # This part is a bit tricky .. I need to parse the injection coordinates and find one-to-one correspondance
        # between cell-types and coordinates
        wb = xlrd.open_workbook('saves/Supplementary_Table_6_Injection_Coordinates.xlsx')
        ws = wb.sheet_by_index(0)
        OutDict2 = {}
        for row_id in range(ws.nrows):
            row =  ws.row(row_id) # get row elements
            if row_id == 0: # do the keys
                OutDict2 = OrderedDict((r.value, []) for r in row)
                key_list = OutDict2.keys()
            else:  # put items in the corresponding keys
                for i,r in enumerate(row): OutDict2[key_list[i]].append(r.value)
                 
            
    return OutDict    

In [1]:
import xml.etree.ElementTree as ET
def Genes_To_Brain_v2(structure, basic_areas):
    
    prim_ids = {structure['id'][idx]:val for idx,val in enumerate(structure['acronym']) for val2 in basic_areas if val == val2}
    Gene_Diff = {}
    page_limit = 12000
    grey = [structure['id'][idx]  for idx,val in enumerate(structure['acronym']) if val == 'grey'][0]
    for prim_id,prim_name in prim_ids.items():
        print(prim_id,prim_name)
        diff_exp_url = 'http://api.brain-map.org/api/v2/data/query.xml?criteria=service::mouse_differential[set$eq{}][structures1$eq{}][structures2$eq{}]'.format('mouse_coronal',prim_id,grey)
        start_page = 0
        end_page = 2000
        while (start_page <= page_limit):
            pagedUrl = diff_exp_url + '[start_row$eq{}][num_rows$eq{}]'.format(start_page,end_page)
            source = urllib.urlopen(pagedUrl).read()
            tree = ET.fromstring(source)
            for tmp1,tmp2 in zip(tree.iter('gene-symbol'),tree.iter('fold-change')):
                if tmp1.text not in Gene_Diff.keys():
                    Gene_Diff[tmp1.text] = OrderedDict((x, 0) for x in prim_ids.values())
                Gene_Diff[tmp1.text][prim_name] = tmp2.text   
            start_page += end_page
            end_page += end_page
            
    Gene_Diff2 = pd.DataFrame(Gene_Diff,index = Gene_Diff[Gene_Diff.keys()[0]].keys(), 
                             columns = Gene_Diff.keys(), dtype = np.float32)            
    return Gene_Diff2

In [ ]:
def CGE(GeneExp, area1,area2):  # given the names of two brain areas, this function estimates the average correlated
                       # gene expression (CGE) between them.
    annotation = nib.load('annotation_200.nii.gz').get_data()
    with open('../CCP_Collab_Backup_2/structures.csv','rb') as fp:
        structure       = PL.readtable(fp) 
    
    mask_path = 'data/annotation200_mask.nii.gz'
    n_mask          = nib.load(os.path.join(mask_path)).get_data() 
    mask2d          = np.reshape(n_mask,[n_mask.shape[0]*n_mask.shape[1]*n_mask.shape[2],1], order='F')
    Non_zero_voxels = ~np.all(mask2d == 0, axis=1)
    flat_annot = np.reshape(annotation,(n_mask.shape[0]*n_mask.shape[1]*n_mask.shape[2]), order = 'F')
    flat_annot = flat_annot[Non_zero_voxels]
    all_ids = np.unique(flat_annot)
    all_ids = np.delete(all_ids,0)
    
    i = 0
    struct_idx_dict = OrderedDict()
    while True:
        all_parent_nodes = GetAncestor(all_ids, level = i, mode = 'id') 
        if area1 not in all_parent_nodes.values(): 
            i +=1
        else:
            break
    for key,val in all_parent_nodes.items():
        if val not in struct_idx_dict.keys():
            struct_idx_dict[val] = []
        struct_idx_dict[val].append(key)   
     
    area1_mask = []
    for val in struct_idx_dict[area1]:
        area1_mask.extend(np.nonzero(flat_annot == val)[0])
    
    i = 0
    struct_idx_dict = OrderedDict()
    while True:
        print(i)
        all_parent_nodes = GetAncestor(all_ids, level = i, mode = 'id') 
        if area2 not in all_parent_nodes.values(): 
            i +=1
        else:
            break
    for key,val in all_parent_nodes.items():
        if val not in struct_idx_dict.keys():
            struct_idx_dict[val] = []
        struct_idx_dict[val].append(key) 
     
    area2_mask = []
    for val in struct_idx_dict[area2]:
        area2_mask.extend(np.nonzero(flat_annot == val)[0])
    
    area1_avg_exp = np.mean(GeneExp[:,area1_mask], axis = 1)
    area2_avg_exp = np.mean(GeneExp[:,area2_mask], axis = 1)
    cge_rho,cge_pval = sci.stats.pearsonr(area1_avg_exp,area2_avg_exp)
    
    return cge_rho,cge_pval
    
    

In [ ]:
def RunFLICA( num_components = 200, brain_data_main_folder = "/home/ntimonidis//knoppers/Nestor/FOR FLICA/",
             output_dir = '/home/ntimonidis/Brain-Connectivity Project/VolumetricAnalysis/data/Outputs_O_param_200/'):
    toolbox_path = "/home/ntimonidis/Brain-Connectivity Project/VolumetricAnalysis/"
    maxits = 3000  # set to 3000 or somethng high, you can check convergence looking at Convergence rate.txt file saved in results.
    lambda_dims = 'o' # 'R' or 'o'  # 'o' encodes modality-wise noise, 'R' encodes modality and subject wise (standard 'o')
    fsl_path ='/usr/local/fsl'
    fs_path ='/Applications/freesurfer'
    tol = 0.000001 #tolerance for convergence. 
    M = flica(brain_data_main_folder, output_dir , num_components, maxits, tol, \
              lambda_dims , fs_path, fsl_path, "PCA")
    return M

In [ ]:
def VolumeLoader(analysis = 'volumetric', infolder = 'data/Outputs_O_param_200/', 
                 indir = '/home/ntimonidis/knoppers/Nestor/venv/FOR FLICA/',
                 twohundred_path = '../25 3 2019/annotation/ccf_2017'
                 ):
    InputData = []
    if analysis == 'unionized':
        ExpPerInj    = h5py.File('G_Exp.hdf5', 'r')['dataset1']
        ExpPerInj    = ExpPerInj[MesoPred.params['leaf_keys'],:] 
        ProjPerInj   = MesoPred.ConDict['wild_type']['ConMat'][MesoPred.params['leaf_keys'],:]
        ExpPerInj,b,ProjPerInj,d = MesoPred.PreProcessing(ExpPerInj, MesoPred.params, ProjPerInj)
        InputData = [ExpPerInj,ProjPerInj]
    elif analysis == 'volumetric':
        for sub_dir in sorted(os.listdir(indir)):
            sub_path = os.path.join(indir,sub_dir)
            for subfiles in os.listdir(sub_path):
                mod_file = os.path.join(sub_path,subfiles)
                InputData.append(nib.load(mod_file).get_data())
                print(InputData[len(InputData)-1].shape)
        
    M = pk.load(open(os.path.join(infolder,'M.p'),'rb')) 
    M['W'] = np.asarray(M['W'])  
    annotation200   = nrrd.read(os.path.join(twohundred_path,'annotation_200.nrrd'))
    average_template_200 = nrrd.read(os.path.join(twohundred_path,'average_template_200.nrrd'))
    shape_of_200       = annotation200[0].shape
    annot_200_res      = np.reshape(annotation200[0],
                                       (shape_of_200[0]*shape_of_200[1]*shape_of_200[2]),order='F')
    template_200_res   = np.reshape(average_template_200[0],
                                       (shape_of_200[0]*shape_of_200[1]*shape_of_200[2]),order='F')
    
    
    return M, InputData, annotation200, annot_200_res

In [1]:
def DocumentThresholdMaps(comp, folder2check, annotation200, low = 1, high = 99, category_list = None,
                         script = './colorcode.sh', mask_list = None, minimum = 30, mode = 'counts'):
    
    turn = 0
    count_dict = OrderedDict()
    ccode_dict_again = OrderedDict()
    allMasks = []
    count_array = np.unique(annotation200[0], return_counts = True)
    all_ids = count_array[0]
    annot_shape = annotation200[0].shape
    annot2d     = (annot_shape[0]*annot_shape[1]*annot_shape[2],1)
    flat_annot = np.ndarray.flatten(annotation200[0],order = 'F')
    all_ids = np.delete(all_ids,0)
    all_parent_nodes = GetAncestor(all_ids, level = 1, mode = 'id')  
    call([script,str(low),str(high),folder2check,str(comp)])
    for mask in os.listdir(os.path.join(folder2check,'Colorcoding_experiments/')):
        mask_key = mask.split('.nii')[0]
        if mask_list is not None and mask_key not in mask_list: continue
        allMasks.append(nib.load(os.path.join(folder2check,'Colorcoding_experiments',mask)).get_data())
        ccode_dict_again[mask_key] = OrderedDict()
    
    for category in ccode_dict_again.keys():
        for node in all_parent_nodes.values():
            ccode_dict_again[category][node] = 0

    for annot_id in range(len(count_array[0])):
        count_dict[count_array[0][annot_id]] = count_array[1][annot_id]
    for category in ccode_dict_again.keys():
        blue_counts = np.asarray(np.unique(annotation200[0][np.nonzero(allMasks[turn])],
                                               return_counts = True))
        primary_src_acros = {key:all_parent_nodes[key] for key in blue_counts[0,:]\
                                 if key in all_parent_nodes.keys()}

        turn+=1
        for structure in np.unique(primary_src_acros.values()):
            related_id_freq = np.asarray([(blue_counts[1, blue_counts[0,:] == key][0],\
                                            len(np.nonzero(annotation200[0] == key)[0]))\
                                            for key,value in primary_src_acros.items() 
                                            if value == structure])
            ccode_dict_again[category][structure] = np.sum(related_id_freq[:,0])
            if mode == 'freq':
                ccode_dict_again[category][structure] = ccode_dict_again[category][structure]/(1.0*np.sum(related_id_freq[:,1]))

    ccode_dict_again['label'] = OrderedDict()
    if category_list == None: category_list = [key for key in ccode_dict_again.keys() if key !='label']
    for structure in all_parent_nodes.values():
        local_vals = [ccode_dict_again[category][structure] for category in category_list]
        max_id = np.argmax(local_vals)
        if np.max(local_vals) > 0: # Proxy built to make sure that labels are assigned based on non-zero values, zero-valued cases shall be labeled as 'other'
            ccode_dict_again['label'][structure] = ccode_dict_again.keys()[max_id]
        else:    
            ccode_dict_again['label'][structure] = 'other'

    return ccode_dict_again

In [ ]:
def ColorcodingRoutine(InputData,annotation200, thr = 10):
    from subprocess import call
    LoadDict = OrderedDict([('data/Outputs_O_param_source_vis_all/',[0,2,4,5,7,8]),\
                ('data/Outputs_O_param_200_source_cp/',[0,3,5]),\
                ('data/Outputs_O_param_200_source_mrn/', [0,4]),
                ('data/Outputs_O_param_200/', [0,1,2,5,10,23])]) 
     
    total_ccode_dict = OrderedDict()
    Tasic_ovp_dict = OrderedDict()
    mix_dict_test = OrderedDict()
    tractography_dict = OrderedDict()
    mix_dict = OrderedDict()
    
    Tasic_ovp_df = []
    Tasic_19_dict = Genes_To_Brain('tasic_19')
    
    all_ids = np.unique(annotation200[0])
    annot_shape = annotation200[0].shape
    annot2d     = (annot_shape[0]*annot_shape[1]*annot_shape[2],1)
    flat_annot = np.ndarray.flatten(annotation200[0],order = 'F')
    all_ids = np.delete(all_ids,0)
    all_parent_nodes = GetAncestor(all_ids, level = 1, mode = 'id')  
    affine_res  = 0.2*np.eye(4); affine_res[3,3] = 1.0
    up = 99; down = 1
    high_perc = 100 - (100 - 97.5)/2; low_perc = (100 - 97.5)/2

    for folder2check, comps_to_check in LoadDict.items():
        M_loc = pk.load(open(os.path.join(folder2check,'M.p'),'rb'))
        M_loc['W'] = np.asarray(M_loc['W'])
        mixed_map = nib.load(os.path.join(folder2check,'niftiOut_mi1.nii.gz')).get_data()
        if folder2check != 'data/Outputs_O_param_200/':                        
            injection_group = folder2check.split('source_')[1].split('/')[0]
            new_mask = pk.load(open('saves/masks_4_wild_type_{}.pkl'.format(injection_group),'rb'))
        else:
            injection_group = 'global'
            new_mask = None

        tmp_list_tmp = []
        for comp in comps_to_check:
            if folder2check != 'data/Outputs_O_param_200/':                        
                injection_group = folder2check.split('source_')[1].split('/')[0]
            else:
                injection_group = 'global'            
            final_key = injection_group + '_' + str(comp)
            final_key = final_key.replace('vis_all','Visual group').replace('cp','Caudoputamen group\
            ').replace('mrn', 'Midbrain group').replace('_',' ').replace('global', 'Global')
            Tasic_ovp_dict[final_key] = OrderedDict([('Gene Markers',[]),('Pathway',[]),('ICA',[])])
            
            print(final_key)
            call(['./colorcode.sh','1','99',folder2check,str(comp)])
            GreenMask = nib.load(os.path.join(folder2check,'Colorcoding_experiments/con_exp_mix_join.nii.gz')).get_data()
            RedMask = nib.load(os.path.join(folder2check,'Colorcoding_experiments/con_dis.nii.gz')).get_data()
            BlueMask = nib.load(os.path.join(folder2check,'Colorcoding_experiments/exp_dis.nii.gz')).get_data()
            MixMapMask = nib.load(os.path.join(folder2check,'Colorcoding_experiments/mixed_maps_thr.nii.gz')).get_data()
            YellowMask = nib.load(os.path.join(folder2check,'Colorcoding_experiments/mixed_maps_bin.nii.gz')).get_data()
            TractMapMask = nib.load(os.path.join(folder2check,'Colorcoding_experiments/sum_con_thr.nii.gz')).get_data()
            allMasks = [GreenMask,BlueMask,RedMask,MixMapMask,TractMapMask]#, YellowMask]
            
            ccode_dict_again = OrderedDict()
            ccode_dict_again['green dict'] = OrderedDict(); ccode_dict_again['blue dict'] = OrderedDict()
            ccode_dict_again['red dict'] = OrderedDict()
            ccode_dict_again['sum dict'] = OrderedDict()
            ccode_dict_again['tract dict'] = OrderedDict()
            #ccode_dict_again['yellow dict'] = OrderedDict()
            for category in ccode_dict_again.keys():
                for node in all_parent_nodes.values():
                    ccode_dict_again[category][node] = 0
            
            turn = 0
            count_dict = OrderedDict()
            count_array = np.unique(annotation200[0],return_counts = True)
            for annot_id in range(len(count_array[0])):
                count_dict[count_array[0][annot_id]] = count_array[1][annot_id]
                
            for category in ccode_dict_again.keys():
                blue_counts = np.asarray(np.unique(annotation200[0][np.nonzero(allMasks[turn])],
                                                       return_counts = True))
                primary_src_acros = {key:all_parent_nodes[key] for key in blue_counts[0,:]\
                                         if key in all_parent_nodes.keys()}
                    
                turn+=1
                for structure in np.unique(primary_src_acros.values()):

                    related_id_freq = np.asarray([(blue_counts[1, blue_counts[0,:] == key][0],\
                                                       len(np.nonzero(annotation200[0] == key)[0]))\
                                                      for key,value in primary_src_acros.items() 
                                                      if value == structure])
                    ccode_dict_again[category][structure] = np.sum(related_id_freq[:,0])
               

                        
            ccode_dict_again['label'] = OrderedDict()
            for structure in all_parent_nodes.values():
                local_vals = [ccode_dict_again[category][structure] for category in \
                              ['green dict', 'blue dict','red dict']]
                max_id = np.argmax(local_vals)
                if np.max(local_vals) > 0:     # Proxy built to make sure that labels are assigned based on non-zero values, zero-valued cases shall be labeled as 'other'
                    ccode_dict_again['label'][structure] = ccode_dict_again.keys()[max_id]
                else:    
                    ccode_dict_again['label'][structure] = 'other'
            
            total_ccode_dict[final_key] = pd.DataFrame.from_dict(ccode_dict_again).sort_values(by = 'green dict', \
                                                                                               ascending = False)
            tractography_dict[final_key] = [key for key,value in ccode_dict_again['tract dict'].items() if value > thr]
            mix_dict[final_key] = [key for key,value in ccode_dict_again['sum dict'].items() if value > thr] 
            tractography_dict[final_key] = np.unique(pd.DataFrame.from_dict(tractography_dict[final_key]).sort_index().values)
            mix_dict[final_key] = np.unique(pd.DataFrame.from_dict(mix_dict[final_key]).sort_index().values)
            mix_dict_test[final_key] = ccode_dict_again['sum dict'] 
            
            sel_gene_footprint = M_loc['X'][0][:,comp]
            gene_high_thr = np.percentile(sel_gene_footprint, high_perc); 
            gene_low_thr = np.percentile(sel_gene_footprint, low_perc)
            gene_grp = [idx for idx,val in enumerate(sel_gene_footprint) 
                            if val >= gene_high_thr or val <= gene_low_thr]
            gene_acro = [MesoPred.params['Gene Acronyms Original'][val] for val in gene_grp 
                             if MesoPred.params['Gene Acronyms Original'][val] is not None]
            unique_set = np.unique(gene_acro)
            
            for idx,cluster in enumerate(Tasic_19_dict['cluster_label']):
                gene_list2 = Tasic_19_dict['markers_global_specific'][idx].split(',') + \
                Tasic_19_dict['markers_subclass_specific'][idx].split(',') + \
                Tasic_19_dict['markers_node_specific'][idx].split(',')
                gene_list2_ovrlp = [gene for gene in gene_list2 if gene in unique_set] 
                if len(gene_list2_ovrlp) > 0:
                    pathway = Tasic_19_dict['class_label'][idx]
                    for g_sub in gene_list2_ovrlp:
                        if g_sub not in Tasic_ovp_dict[final_key]['Gene Markers']:
                            Tasic_ovp_dict[final_key]['Gene Markers'].extend([g_sub])
                            Tasic_ovp_dict[final_key]['Pathway'].extend([pathway])
                            Tasic_ovp_dict[final_key]['ICA'].extend([key])

            Tasic_ovp_df.append(pd.DataFrame.from_dict(Tasic_ovp_dict[final_key]))
    
    SummaryDict = OrderedDict()
    for inj_group in total_ccode_dict.keys():
        sub_group = [] 
        for label in total_ccode_dict[inj_group].keys():
            if label == 'label' or label == 'yellow dict': continue
            key = inj_group + ' ' + label
            key = key.replace('blue dict',\
                              'gene expression driven').replace('red dict',\
                                                                'projection density driven').replace('green dict', 'bimodal')
            SummaryDict[key] = total_ccode_dict[inj_group].loc[total_ccode_dict[inj_group]['label'] == label].loc\
            [total_ccode_dict[inj_group][label] > thr].sort_index().index
    
    MixDict_test_df = pd.DataFrame(mix_dict_test,index = mix_dict_test[list(mix_dict_test.keys())[0]].keys(),\
                                              columns= mix_dict_test.keys()).T
    
    return SummaryDict, Tasic_ovp_df, mix_dict, tractography_dict, MixDict_test_df, total_ccode_dict

In [2]:
def BuildTargetMask(target_name = 'Primary visual area'):
    mask_path = 'data/annotation200_mask.nii.gz'
    n_mask = nib.load(os.path.join(mask_path)).get_data() 
    mask2d = np.reshape(n_mask,[n_mask.shape[0]*n_mask.shape[1]*n_mask.shape[2],1], order='F')
    Non_zero_voxels = ~np.all(mask2d == 0, axis = 1)
    annotation200 = nrrd.read('../25 3 2019/annotation/ccf_2017/annotation_200.nrrd')
    flat_200 = np.ndarray.flatten(annotation200[0], order = 'F')
    flat_200 = flat_200[Non_zero_voxels]
    all_ids = np.unique(flat_200)
    all_ids = np.delete(all_ids,0)
    all_parent_nodes = GetAncestor(all_ids, level = 1, mode = 'id') 
    ids_of_int = [key for key,value in all_parent_nodes.items() if value == target_name]
    vox_locs = np.unique([val2 for val in ids_of_int for val2 in np.where(flat_200 == val)[0]])
    return vox_locs


In [ ]:
def BuildInjectionSubsets(Data_Modality, sources, group_num = 3, sources2 = None, \
                          Data_Modality2 = None, sources3 = None, Data_Modality3 = None,
                          filename = None, annotation200 = None, target = 'motor',
                          cell_specificity = None): 
    # sources 2 is the latest addition in case you want to mark brain areas for a second modality
    # i.e. DTI or morphologies
    # latest addition: cell-specificity for projections originating from specific cell classes
    ## Isolate injections heavily clustered in one ROI, build a mask, do local flica on them
    mask_path        = 'data/annotation200_mask.nii.gz'
    n_mask           = nib.load(os.path.join(mask_path)).get_data() 
    mask2d           = np.reshape(n_mask,[n_mask.shape[0]*n_mask.shape[1]*n_mask.shape[2],1], order='F')
    Non_zero_voxels  = ~np.all(mask2d == 0, axis=1)
    annot_nzero      = []
    if annotation200 is not None:
        flat_annot       = np.ndarray.flatten(annotation200[0],order = 'F') 
        annot_nzero      = flat_annot[Non_zero_voxels]
    if cell_specificity is not None:
        NeuronCT         = pk.load(open('../Mouselight/saves/Neuron_CT.pkl','rb'))
    new_masks = OrderedDict()

    unique_srcs = np.unique(sources)
    cls_membs = [] 
    for src in unique_srcs:
        cls_membs.append([idx for idx,val in enumerate(sources) if val == src])
      
    # get the most frequent source 
    inj_freq = [len(val) for val in cls_membs]
    top_cls = np.argsort(inj_freq)[::-1]
    inj_grps = OrderedDict()
    inj_grps['vis_all'] = np.unique([idx for idx,val in enumerate(unique_srcs) for val2 in cls_membs[idx] \
                      if 'VIS' in val and val !='VISC'])
    inj_grps['MO_all'] = np.unique([idx for idx,val in enumerate(unique_srcs) for val2 in cls_membs[idx] \
                      if 'MOp' in val or 'MOs' in val])
     
    for selection in range(1,group_num): 
        inj_grps[unique_srcs[top_cls[selection]]] = [top_cls[selection]]
    
    # make a mask based on it 
    for inj_name,inj_grp in inj_grps.items():
        new_masks[inj_name] = {}
        src_mask = np.zeros(np.shape(sources))
        if inj_name == 'vis_all':
            new_masks[inj_name]['source_name'] = inj_name
            for val in inj_grp:
                src_mask[cls_membs[val]] = 1
            if sources2 is not None:
                new_masks[inj_name]['source_mask_II'] = [neuron_id for neuron_id,neuron_name in enumerate(sources2) \
                                               if 'VIS' in neuron_name and neuron_name !='VISC']
            if sources3 is not None:
                new_masks[inj_name]['source_mask_III'] = [neuron_id for neuron_id,neuron_name in enumerate(sources3) \
                                               if 'VIS' in neuron_name and neuron_name !='VISC']
        elif inj_name == 'MO_all':
            new_masks[inj_name]['source_name'] = inj_name
            for val in inj_grp:
                src_mask[cls_membs[val]] = 1
            if sources2 is not None:
                if cell_specificity is not None:
                    new_masks[inj_name]['source_mask_II'] = [neuron_id for neuron_id,neuron_name in \
                                                             enumerate(sources2) if 'MOp' in neuron_name \
                                                             or 'MOs' in neuron_name and neuron_id in NeuronCT.keys() \
                                                             and NeuronCT[neuron_id] == cell_specificity]     
                else:
                    new_masks[inj_name]['source_mask_II'] = [neuron_id for neuron_id,neuron_name in \
                                                             enumerate(sources2) \
                                               if 'MOp' in neuron_name or 'MOs' in neuron_name] 
            if sources3 is not None:
                new_masks[inj_name]['source_mask_III'] = [neuron_id for neuron_id,neuron_name in \
                                                              enumerate(sources3) if 'MOp' in neuron_name \
                                                              or 'MOs' in neuron_name]     
        else: 
            new_masks[inj_name]['source_name'] = unique_srcs[inj_grp]
            src_mask[cls_membs[inj_grp[0]]] = 1  
            if sources2 is not None:
                new_masks[inj_name]['source_mask_II'] = [neuron_id for neuron_id,neuron_name in enumerate(sources2) \
                                               if neuron_name == inj_name]
            if sources3 is not None:
                new_masks[inj_name]['source_mask_III'] = [neuron_id for neuron_id,neuron_name in enumerate(sources3) \
                                               if neuron_name == inj_name]    
                
        new_masks[inj_name]['source_mask'] = np.nonzero(src_mask)[0]
        
        # Find a common ground for those injections
        tmp_mat = Data_Modality[new_masks[inj_name]['source_mask'],:]
        BinMat  = np.zeros(np.shape(tmp_mat))
        min_el  = np.min(BinMat)
        BinMat[tmp_mat != min_el] = 1
        common_vox = np.asarray(np.prod(BinMat, axis = 0))
        new_masks[inj_name]['vox_mask'] = np.nonzero(common_vox)[0]
        
        if annotation200 is not None:
            tmp_mat2 = tmp_mat + np.min(tmp_mat) + 1e-10
            log_proj = np.log10(tmp_mat2)
            npv_mask = []
            for proj in range(len(log_proj[0])):
                #low_iqr = np.median(log_proj[:,proj]) - 1.5*sci.stats.iqr(log_proj[:,proj])
                low_iqr = np.median(log_proj[:,proj])
                if low_iqr > -1.5:
                    npv_mask.append(proj)

            annot_nzero_mask = annot_nzero[npv_mask]
            all_ids          = np.unique(annot_nzero_mask)
            if len(annot_nzero_mask) > 0:
                all_parent_nodes = GetAncestor(all_ids, level = 1, mode = 'id')
                new_masks[inj_name]['target_mask'] = np.unique(all_parent_nodes.values())

        if Data_Modality2 is not None and 'source_mask_II' in new_masks[inj_name].keys() and \
        len(new_masks[inj_name]['source_mask_II']) > 0:
            tmp_mat = Data_Modality2[new_masks[inj_name]['source_mask_II'],:]
            BinMat  = np.zeros(np.shape(tmp_mat))
            min_el  = np.min(BinMat)
            BinMat[tmp_mat != min_el] = 1
            common_vox2 = np.asarray(np.sum(BinMat, axis = 0))
            new_masks[inj_name]['vox_mask'] = np.nonzero(common_vox2)[0]
            #new_masks[inj_name]['vox_mask'] = np.unique(np.concatenate((new_masks[inj_name]['vox_mask'], \
            #                                                            np.nonzero(common_vox2)[0])))
            if annotation200 is not None:
                annot_nzero_mask = annot_nzero[np.nonzero(common_vox2)[0]]
                all_ids          = np.unique(annot_nzero_mask)
                all_parent_nodes = GetAncestor(all_ids, level = 1, mode = 'id')
                new_masks[inj_name]['target_mask_2'] = np.unique(all_parent_nodes.values())
            
            
        if Data_Modality3 is not None and 'source_mask_III' in new_masks[inj_name].keys() and \
        len(new_masks[inj_name]['source_mask_III']) > 0:
            tmp_mat = Data_Modality3[new_masks[inj_name]['source_mask_III'],:]
            BinMat  = np.zeros(np.shape(tmp_mat))
            min_el  = np.min(BinMat)
            BinMat[tmp_mat != min_el] = 1
            common_vox3 = np.asarray(np.prod(BinMat, axis = 0))
            #new_masks[inj_name]['vox_mask'] = np.nonzero(common_vox2)[0]
            new_masks[inj_name]['vox_mask'] = np.unique(np.concatenate((new_masks[inj_name]['vox_mask'], \
                                                                        np.nonzero(common_vox3)[0])))  
        
        print inj_name, len(new_masks[inj_name]['source_mask']), len(new_masks[inj_name]['vox_mask'])
                
        # Here I summon the allen annotation of 200, I reduce structures to parental ones and get their 
        # unique set, which will be stored as a key
        # I shall repeat this for gene expression        
        if 'source_mask_II' in new_masks[inj_name].keys():   
            print len(new_masks[inj_name]['source_mask_II'])
        if 'source_mask_III' in new_masks[inj_name].keys():
            print len(new_masks[inj_name]['source_mask_III'])
            
            
        if filename is not None:    
            if cell_specificity is not None:
                pk.dump(new_masks[inj_name], open(os.path.join('saves', filename + '_{}.pkl'.\
                                                               format(inj_name + ' ' + cell_specificity)),'wb'))
            else:    
                pk.dump(new_masks[inj_name], open(os.path.join('saves', filename + '_{}.pkl'.format(inj_name)),'wb'))

    return new_masks

In [ ]:
def PairwiseVoxelCorrelation(comp_set, lica_file = 'data/Outputs_O_param_200', dict_file = 'temp_saves/'):
    
    import flica_various as alb_various
    G_code = pk.load(open(os.path.join(dict_file, 'Atoms_4_GeneVol.pkl'),'rb')).T
    P_code = pk.load(open(os.path.join(dict_file, 'Atoms_4_ProjVol.pkl'),'rb')).T
    G_atom = pk.load(open(os.path.join(dict_file, 'Code_4_GeneVol.pkl'),'rb')).T
    P_atom = pk.load(open(os.path.join(dict_file, 'Code_4_ProjVol.pkl'),'rb')).T
    CC_atom = pk.load(open(os.path.join(dict_file, 'Atoms_4_GeneVol_conc.pkl'),'rb'))
    CC_code = pk.load(open(os.path.join(dict_file, 'Code_4_GeneVol_conc.pkl'),'rb'))

    LoadDict = OrderedDict([(lica_file,comp_set)]) 
    rounding_to_avoid = ['Source to LICA p','Spatial map p', 'Gene to LICA p']

    InputData = [G_code,P_code]
    dictsize = len(G_atom)
    Dict2Dict_dict = OrderedDict()
    Dict2Loc_dict = OrderedDict()
    Dict2Dict_dict = OrderedDict()
    LICA_2_dict = OrderedDict()
    LICA_2_dict2 = OrderedDict()
    LICA_2_dict = np.zeros((dictsize,dictsize))
    Dict2Dict = np.zeros((dictsize,dictsize))
    Dict2Dict2 = np.zeros((dictsize,dictsize))
    Dict2Loc = np.zeros((dictsize,dictsize))
    Dict2Loc2 = np.zeros((dictsize,dictsize))

    M = pk.load(open(os.path.join(lica_file,'M.p'),'rb')) 
    injection_group = 'global'
    M['W'] = np.asarray(M['W'])
    G2_code = CC_code[0:3318,:]; P2_code = CC_code[3318:len(CC_code),:] 
    pc=np.divide(M['H_PCs'][0:-1][:],np.tile(np.sum(M['H_PCs'][0:-1][:],0),[M['H_PCs'].shape[0]-1, 1] ))
    pc = np.asarray(pc)
    comps_of_int = [0,1,2,4,5,10,23]
    compsize = len(comps_of_int)
    top_num = 0

    Dict_Modality_sc = {}
    k = 0
    for mod in InputData:
        Dict_Modality_sc[k] = InputData[k] - (np.matrix(np.mean(InputData[k],1)).T)       
        scaling_data_transform = alb_various.rms(Dict_Modality_sc[k],[],[])       
        Dict_Modality_sc[k] = np.divide(Dict_Modality_sc[k],scaling_data_transform)
        Dict_Modality_sc[k] = np.asarray(Dict_Modality_sc[k])
        k+=1

    for dicti in range(dictsize):
        for l_i in comps_of_int:
            Dict2Loc[l_i,dicti] = sci.stats.pearsonr(np.abs(M['H'][l_i,:]),\
                                                     np.abs(Dict_Modality_sc[0][:,dicti]))[0]
            Dict2Loc2[l_i,dicti] = sci.stats.pearsonr(np.abs(M['H'][l_i,:]),\
                                                      np.abs(Dict_Modality_sc[1][:,dicti]))[0]
        for dictj in range(dictsize):
            Dict2Dict[dicti,dictj] = sci.stats.pearsonr(np.abs(Dict_Modality_sc[0][:,dicti]),\
                                                         np.abs(Dict_Modality_sc[1][:,dictj]))[0]
            LICA_2_dict[dicti,dictj] = sci.stats.pearsonr(np.abs(M['H'][dicti,:]),\
                                                         np.abs(CC_atom[dictj,:]))[0]    

    for comp in comps_of_int:
        print(comp)
        top_num_to_pick = 0
        new_key = '{} ICA {}'.format(injection_group, comp)
        #max_glob_id = np.argmax(Dict2Loc[comp,:])
        max_glob_id = np.argsort(Dict2Loc[comp,:])[::-1][top_num_to_pick]
        #max_glob_val = np.max(Dict2Loc[comp,:])
        max_glob_val = np.sort(Dict2Loc[comp,:])[::-1][top_num_to_pick]
        max_glob_p   = sci.stats.pearsonr(np.abs(M['H'][comp,:]),\
                                          np.abs(Dict_Modality_sc[0][:,max_glob_id]))[1]

        max_glob_id2 = np.argsort(Dict2Loc2[comp,:])[::-1][top_num_to_pick]
        max_glob_val2 = np.sort(Dict2Loc2[comp,:])[::-1][top_num_to_pick]
        max_glob_p2   = sci.stats.pearsonr(np.abs(M['H'][comp,:]),\
                                           np.abs(Dict_Modality_sc[1][:,max_glob_id2]))[1]

        gene_coef_cor = sci.stats.pearsonr(np.abs(M['X'][0][:,comp]),\
                                           np.abs(G_atom[max_glob_id,:]))
        source_coef_cor = sci.stats.pearsonr(np.abs(M['X'][1][:,comp]),\
                                             np.abs(P_atom[max_glob_id2,:]))

        interdict_cor = sci.stats.pearsonr(np.abs(Dict_Modality_sc[0][:,max_glob_id]),\
                                           np.abs(Dict_Modality_sc[1][:,max_glob_id2]))

        Dict2Loc_dict[new_key] = OrderedDict([('Gene Dictionary', max_glob_id),\
                                              ('A1 rho', max_glob_val),\
                                              ('A1 p', max_glob_p) ,\
                                              ('A2 rho', gene_coef_cor[0]),\
                                              ('A2 p', gene_coef_cor[1]),
                                              ('Projection Dictionary', max_glob_id2),\
                                              ('B1 rho', max_glob_val2), \
                                              ('B1 p', max_glob_p2),\
                                              ('B2 rho', source_coef_cor[0]),\
                                              ('B2 p', source_coef_cor[1]),\
                                              ('interdict rho', interdict_cor[0]), \
                                              ('interdict p', interdict_cor[1])])
        for key2 in Dict2Loc_dict[new_key].keys():
            if key2 in rounding_to_avoid:
                continue # for now
            else:
                Dict2Loc_dict[new_key][key2] = np.around(Dict2Loc_dict[new_key][key2],decimals = 3)

        LICA_2_dict2[comp] = OrderedDict()
        top_conc_dict = np.argsort(LICA_2_dict[comp,:])[::-1][top_num]
        LICA_2_dict2[comp]['Concat Dictionary'] = top_conc_dict
        LICA_2_dict2[comp]['Pearson rho']  = np.sort(LICA_2_dict[comp,:])[::-1][top_num]
        LICA_2_dict2[comp]['Pearson p']    = sci.stats.pearsonr(np.abs(M['H'][comp,:]),np.abs(CC_atom[top_conc_dict,:]))[1]
        LICA_2_dict2[comp]['C']     = sci.stats.pearsonr(np.abs(M['X'][0][:,comp]),np.abs(G2_code[:,top_conc_dict]))[0]
        LICA_2_dict2[comp]['D']   = sci.stats.pearsonr(np.abs(M['X'][1][:,comp]),np.abs(P2_code[:,top_conc_dict]))[0]
        LICA_2_dict2[comp]['E']  = pc[0][comp] 
        
        for tmp_key in LICA_2_dict2[comp].keys():
            LICA_2_dict2[comp][tmp_key] = np.around(LICA_2_dict2[comp][tmp_key],decimals = 3)

    Dict2Loc_df = pd.DataFrame(Dict2Loc_dict,index = Dict2Loc_dict[Dict2Loc_dict.keys()[0]].keys(),\
                               columns = Dict2Loc_dict.keys()).T
    Dict2Loc_df = Dict2Loc_df.astype({'Gene Dictionary': 'int32'}, copy = False)
    Dict2Loc_df = Dict2Loc_df.astype({'Projection Dictionary': 'int32'}, copy = False)

    LICA_2_dict_df = pd.DataFrame(LICA_2_dict2,index = LICA_2_dict2[LICA_2_dict2.keys()[0]].keys(),\
                                   columns = LICA_2_dict2.keys()).T  
    LICA_2_dict_df = LICA_2_dict_df.astype({'Concat Dictionary': 'int32'}, copy = False)
    return Dict2Loc_df, LICA_2_dict_df

In [ ]:
# New additions 25-05-2020: update Component summary by including 
#                          a) spectral clustering of H and then nmi with the allen annotation
#                          b) pearson correlation with the average template volume
#a)
from sklearn.cluster import spectral_clustering, SpectralClustering
from sklearn.metrics import normalized_mutual_info_score as nmi    

def PostHocLICA(annotation200, Non_zero_voxels, infile = 'data/Outputs_O_param_source_vis_all/M.p',
                maskfile = 'saves/masks_4_wild_type_vis_all.pkl',
                tempfile = 'average_template_200.nii.gz',
                comp_filter = None):

    M = pk.load(open(infile,'rb'))
    new_masks = pk.load(open(maskfile,'rb'))
    avg_template = nib.load(tempfile).get_data()
    lica2gray = np.zeros((len(M['H']),2))
    flat_avg_temp = np.ndarray.flatten(avg_template,order = 'F')
    flat_annot = np.ndarray.flatten(annotation200[0],order = 'F')
    
    if comp_filter is None:
        comp_filter = np.arange(len(M['H']))
    
    affinity_2d = np.corrcoef(M['H'][comp_filter,:].T)
    spect_clus  = SpectralClustering(n_clusters=20, affinity = 'precomputed').fit(np.abs(affinity_2d))
    labels      = spect_clus.labels_
    nmi_score   = nmi(flat_annot[Non_zero_voxels][new_masks['vox_mask']],labels)
    print np.shape(labels), nmi_score 

    flat_avg_temp_sub = flat_avg_temp[Non_zero_voxels][new_masks['vox_mask']]
    flat_avg_temp_sub_z = sci.stats.zscore(flat_avg_temp_sub)
    for i in comp_filter:
        lica2gray[i,:] = sci.stats.pearsonr(M['H'][i,:],flat_avg_temp_sub_z)

    plt.hist(lica2gray)    


    return lica2gray, labels, nmi_score